# 1. Setup and Load Data

## 1.1 Install Dependencies and Setup

In [ ]:
!pip install tensorflow opencv-python matplotlib

In [ ]:
!pip install python-magic

In [ ]:
!pip list

In [ ]:
import tensorflow as tf
import os

In [ ]:
os.listdir('/kaggle/input/cifar10/cifar10')

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
gpus

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

## 1.2 Filter Data

In [ ]:
import cv2
import magic

In [ ]:
data_dir = '/kaggle/input/cifar10/cifar10/train'

In [ ]:
os.listdir(data_dir)

In [ ]:
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [ ]:
image_exts

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            # Use python-magic to determine the file type
            tip = magic.from_file(image_path, mime=True)
            # Extract the extension from the MIME type
            tip = tip.split('/')[-1]
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)

        except Exception as e:
            print('Issue with image {}'.format(image_path))

## 1.3 Load Data 

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('/kaggle/input/cifar10/cifar10/train')

In [ ]:
# Assuming 'train' is the name of your training dataset object
class_names = data.class_names
print(class_names)

In [ ]:
# 1. Get a new iterator and batch from the NORMALIZED data pipeline
data_iterator = data.as_numpy_iterator()

In [ ]:
new_batch = data_iterator.next()

In [ ]:
# 2. Plot the new batch
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(new_batch[0][:4]):
    # The image is already a float in the [0, 1] range, so imshow handles it perfectly
    ax[idx].imshow(img)
    ax[idx].title.set_text(new_batch[1][idx])

# 2 Preprocess Data

## 2.1 Scale Data

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()[0].max()

In [ ]:
scaled_batch = data.as_numpy_iterator().next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(scaled_batch[0][:4]):
    # 'img' is now a float in the [0, 1] range, and imshow will work perfectly
    ax[idx].imshow(img)
    ax[idx].title.set_text(scaled_batch[1][idx])

## 2.2 Split Data

In [ ]:
len(data)

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
train_size+val_size+test_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# 3. Deep Model

## 3.1 Build Deep Learning Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

In [ ]:
data_augmentation = Sequential([
    RandomFlip('horizontal'),
    RandomRotation(0.1),
    RandomZoom(0.1),
])

In [ ]:
model = Sequential([
    # Input Layer and Convolutional Base
    Conv2D(16, kernel_size=(3,3), strides=1, activation='relu', input_shape=(256,256,3)),
    MaxPooling2D(),
    
    Conv2D(32, kernel_size=(3,3), strides=1, activation='relu'),
    MaxPooling2D(),
    
    Conv2D(16, kernel_size=(3,3), strides=1, activation='relu'),
    MaxPooling2D(),
    Dropout(0.25),
    
    # Flatten the 4D output to 2D for the Dense layers
    Flatten(),
    
    # Dense (Fully Connected) Head
    # Add L2 regularization with a factor of 0.001
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.25),
    
    Dense(10, activation='softmax') # Use 'softmax' for multi-class classification
])

In [ ]:
model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
# --- Please use this exact code to save your model ---

# 1. Define the full filepath, including the name and the required .keras extension
save_path = '/kaggle/working/cifar10_model.keras'

# 2. Save the model to that specific file
model.save(save_path)

# 3. Print a confirmation message
print(f"Model successfully saved to {save_path}")

## 3.2 Train 

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# Stop training if val_loss doesn't improve for 3 consecutive epochs
early_stopper = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Pass the callback to the fit method
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[early_stopper])

In [ ]:
hist.history

## 3.3 Plot Performance 

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 4. Evaluate Performance

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
loaded_model = load_model('/kaggle/input/cifar/tensorflow2/default/1/cifar10_model.keras')

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch 
    
    # 1. Get the raw probability predictions from the model
    yhat_probs = model.predict(X, verbose=0) # Using verbose=0 to keep the output clean
    
    # 2. Convert the probabilities into the final predicted class index (THE FIX!)
    yhat = tf.argmax(yhat_probs, axis=1)
    
    # 3. Now update your metrics. The shapes of 'y' and 'yhat' will match.
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

## 4.2 Test 

In [ ]:
img = cv2.imread('/kaggle/input/temp2020/pexels-pixabay-104827.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255,0))

In [ ]:
yhat